In [1]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

def load_images_from_folder(folder, label, image_size=(227, 227)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, image_size)  # Resize to match the input size
            img = img / 255.0  # Normalize to [0, 1]
            images.append(img)
            labels.append(label)
    return images, labels

# Paths to your datasets
path_to_stroke_images = r'D:\Coding\A Machine Learning-Based Diagnostic Model Using Neuroimages\Brain_Data_Organised\Stroke'
path_to_normal_images = r'D:\Coding\A Machine Learning-Based Diagnostic Model Using Neuroimages\Brain_Data_Organised\Normal'

# Load stroke and normal images
stroke_images, stroke_labels = load_images_from_folder(path_to_stroke_images, 1)
normal_images, normal_labels = load_images_from_folder(path_to_normal_images, 0)

# Combine data
X = np.array(stroke_images + normal_images)
y = np.array(stroke_labels + normal_labels)

# Expand dimensions to add channel information (1 channel for grayscale)
X = np.expand_dims(X, axis=-1)

# Split the dataset into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [2]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Model

# Load the VGG19 model pre-trained on ImageNet data
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(227, 227, 3))

# Define the layer from which you want to get the output
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)

def extract_features(X):
    # Because VGG19 expects three channels, we need to repeat our grayscale data
    X_repeated = np.repeat(X, 3, axis=3)
    features = model.predict(X_repeated)
    return features

# Extract features
X_train_features = extract_features(X_train)
X_val_features = extract_features(X_val)
X_test_features = extract_features(X_test)

55/55 ━━━━━━━━━━━━━━━━━━━━ 106s 2s/step
12/12 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step
12/12 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step


In [3]:
from sklearn.decomposition import PCA

# Apply PCA to reduce the features' dimensionality
pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train_features.reshape(X_train_features.shape[0], -1))
X_val_pca = pca.transform(X_val_features.reshape(X_val_features.shape[0], -1))
X_test_pca = pca.transform(X_test_features.reshape(X_test_features.shape[0], -1))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout

def build_BiLSTM(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=input_shape))
    model.add(Bidirectional(LSTM(32)))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Reshape for LSTM input
X_train_lstm = X_train_pca.reshape((X_train_pca.shape[0], 1, X_train_pca.shape[1]))
X_val_lstm = X_val_pca.reshape((X_val_pca.shape[0], 1, X_val_pca.shape[1]))
X_test_lstm = X_test_pca.reshape((X_test_pca.shape[0], 1, X_test_pca.shape[1]))

# Build and train the model
model = build_BiLSTM((1, X_train_pca.shape[1]))
history = model.fit(X_train_lstm, y_train, epochs=20, validation_data=(X_val_lstm, y_val))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_lstm, y_test)
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/20


d:\Coding\A Machine Learning-Based Diagnostic Model Using Neuroimages\myenv\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.6107 - loss: 0.6701 - val_accuracy: 0.6507 - val_loss: 0.5856
Epoch 2/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7561 - loss: 0.5025 - val_accuracy: 0.8747 - val_loss: 0.3174
Epoch 3/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9598 - loss: 0.1759 - val_accuracy: 0.9227 - val_loss: 0.1912
Epoch 4/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9897 - loss: 0.0428 - val_accuracy: 0.9413 - val_loss: 0.1358
Epoch 5/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9972 - loss: 0.0163 - val_accuracy: 0.9413 - val_loss: 0.1295
Epoch 6/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9984 - loss: 0.0090 - val_accuracy: 0.9360 - val_loss: 0.1469
Epoch 7/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9949 - loss: 0.0134 - val_accuracy: 0.9387 - val_loss: 0.1433
Epoch 8/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9997 - loss: 0.0029 - val_accuracy: 0.9440 - val_loss: 0.1839
Epo

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Predictions
y_pred = (model.predict(X_test_lstm) > 0.5).astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, model.predict(X_test_lstm))

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"ROC AUC: {roc_auc}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 725us/step
Accuracy: 0.976063829787234
Precision: 0.9632352941176471
Recall: 0.9703703703703703
F1 Score: 0.966789667896679
ROC AUC: 0.9954510527124635
